In [3]:
import json
import os
from io import BytesIO

import eyed3
import ffmpeg
import requests
from yt_dlp import YoutubeDL

In [4]:
# Playlist
youtube_urls = "https://music.youtube.com/playlist?list=PLBRGkzFPYJTm-nBmUmWv8uo5tk9KNQ_9R&si=kUEZrALMydJcRcxR"

options = {
    "listformats": True,
    "ignoreerrors": True,
    "cookiefile": "./outputs/cookies.txt",
}

with YoutubeDL(options) as ydl:
    info = ydl.extract_info(youtube_urls, download=False)

[youtube:tab] Extracting URL: https://music.youtube.com/playlist?list=PLBRGkzFPYJTm-nBmUmWv8uo5tk9KNQ_9R&si=kUEZrALMydJcRcxR


[youtube:tab] PLBRGkzFPYJTm-nBmUmWv8uo5tk9KNQ_9R: Downloading webpage
[youtube:tab] PLBRGkzFPYJTm-nBmUmWv8uo5tk9KNQ_9R: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: tmp
[youtube:tab] PLBRGkzFPYJTm-nBmUmWv8uo5tk9KNQ_9R page 1: Downloading API JSON
[youtube:tab] Playlist tmp: Downloading 40 items of 40
[download] Downloading item 1 of 40
[youtube] Extracting URL: https://music.youtube.com/watch?v=umKLy_E5Dyk
[youtube] umKLy_E5Dyk: Downloading webpage
[youtube] umKLy_E5Dyk: Downloading ios player API JSON
[youtube] umKLy_E5Dyk: Downloading mweb player API JSON
[youtube] umKLy_E5Dyk: Downloading ios music player API JSON
[youtube] umKLy_E5Dyk: Downloading web music client config
[youtube] umKLy_E5Dyk: Downloading player 03dbdfab
[youtube] umKLy_E5Dyk: Downloading web music player API JSON


[youtube] umKLy_E5Dyk: Downloading m3u8 information
[info] Available formats for umKLy_E5Dyk:
ID  EXT   RESOLUTION FPS CH |   FILESIZE   TBR PROTO | VCODEC          VBR ACODEC      ABR ASR MORE INFO
----------------------------------------------------------------------------------------------------------------
sb2 mhtml 48x27        1    |                  mhtml | images                                  storyboard
sb1 mhtml 45x45        1    |                  mhtml | images                                  storyboard
sb0 mhtml 90x90        1    |                  mhtml | images                                  storyboard
233 mp4   audio only        |                  m3u8  | audio only          unknown             Default
234 mp4   audio only        |                  m3u8  | audio only          unknown             Default
139 m4a   audio only      2 |  959.90KiB   49k https | audio only          mp4a.40.5   49k 22k low, m4a_dash
249 webm  audio only      2 |    1.13MiB   59k https | 

[youtube] mFAodH3PjPo: Downloading m3u8 information
[info] Available formats for mFAodH3PjPo:
ID  EXT   RESOLUTION FPS CH |    FILESIZE  TBR PROTO | VCODEC         VBR ACODEC      ABR ASR MORE INFO
---------------------------------------------------------------------------------------------------------------
sb2 mhtml 48x27        0    |                  mhtml | images                                 storyboard
sb1 mhtml 45x45        1    |                  mhtml | images                                 storyboard
sb0 mhtml 90x90        1    |                  mhtml | images                                 storyboard
233 mp4   audio only        |                  m3u8  | audio only         unknown             Default
234 mp4   audio only        |                  m3u8  | audio only         unknown             Default
139 m4a   audio only      2 |     1.33MiB  49k https | audio only         mp4a.40.5   49k 22k low, m4a_dash
249 webm  audio only      2 |     1.58MiB  58k https | audio on

[youtube] 9zeE-ScuVNg: Downloading m3u8 information
[info] Available formats for 9zeE-ScuVNg:
ID  EXT   RESOLUTION FPS CH |   FILESIZE  TBR PROTO | VCODEC         VBR ACODEC      ABR ASR MORE INFO
--------------------------------------------------------------------------------------------------------------
sb3 mhtml 48x27        0    |                 mhtml | images                                 storyboard
sb2 mhtml 45x45        1    |                 mhtml | images                                 storyboard
sb1 mhtml 90x90        1    |                 mhtml | images                                 storyboard
sb0 mhtml 180x180      1    |                 mhtml | images                                 storyboard
233 mp4   audio only        |                 m3u8  | audio only         unknown             Default
234 mp4   audio only        |                 m3u8  | audio only         unknown             Default
139 m4a   audio only      2 |    1.23MiB  49k https | audio only         m

...

[youtube] XOIzYukFl0Q: Downloading m3u8 information
[info] Available formats for XOIzYukFl0Q:
ID  EXT   RESOLUTION FPS CH |   FILESIZE  TBR PROTO | VCODEC         VBR ACODEC      ABR ASR MORE INFO
--------------------------------------------------------------------------------------------------------------
sb3 mhtml 48x27        0    |                 mhtml | images                                 storyboard
sb2 mhtml 45x45        1    |                 mhtml | images                                 storyboard
sb1 mhtml 90x90        1    |                 mhtml | images                                 storyboard
sb0 mhtml 180x180      1    |                 mhtml | images                                 storyboard
233 mp4   audio only        |                 m3u8  | audio only         unknown             Default
234 mp4   audio only        |                 m3u8  | audio only         unknown             Default
139 m4a   audio only      2 |    1.45MiB  49k https | audio only         m

[youtube] Ngww2sWyDpA: Downloading m3u8 information
[info] Available formats for Ngww2sWyDpA:
ID  EXT   RESOLUTION FPS CH |   FILESIZE   TBR PROTO | VCODEC          VBR ACODEC      ABR ASR MORE INFO
----------------------------------------------------------------------------------------------------------------
sb3 mhtml 48x27        0    |                  mhtml | images                                  storyboard
sb2 mhtml 45x45        0    |                  mhtml | images                                  storyboard
sb1 mhtml 90x90        0    |                  mhtml | images                                  storyboard
sb0 mhtml 180x180      0    |                  mhtml | images                                  storyboard
233 mp4   audio only        |                  m3u8  | audio only          unknown             Default
234 mp4   audio only        |                  m3u8  | audio only          unknown             Default
139 m4a   audio only      2 |    2.03MiB   49k https | aud

[youtube] 7UtbITFIzOE: Downloading m3u8 information
[info] Available formats for 7UtbITFIzOE:
ID  EXT   RESOLUTION FPS CH |   FILESIZE   TBR PROTO | VCODEC          VBR ACODEC      ABR ASR MORE INFO
----------------------------------------------------------------------------------------------------------------
sb3 mhtml 48x27        1    |                  mhtml | images                                  storyboard
sb2 mhtml 45x45        1    |                  mhtml | images                                  storyboard
sb1 mhtml 90x90        1    |                  mhtml | images                                  storyboard
sb0 mhtml 180x180      1    |                  mhtml | images                                  storyboard
233 mp4   audio only        |                  m3u8  | audio only          unknown             Default
234 mp4   audio only        |                  m3u8  | audio only          unknown             Default
139 m4a   audio only      2 |  904.61KiB   49k https | aud

[youtube] hXrr9iu1Ois: Downloading m3u8 information
[info] Available formats for hXrr9iu1Ois:
ID  EXT   RESOLUTION FPS CH |   FILESIZE   TBR PROTO | VCODEC          VBR ACODEC      ABR ASR MORE INFO
----------------------------------------------------------------------------------------------------------------
sb2 mhtml 48x27        1    |                  mhtml | images                                  storyboard
sb1 mhtml 45x45        1    |                  mhtml | images                                  storyboard
sb0 mhtml 90x90        1    |                  mhtml | images                                  storyboard
233 mp4   audio only        |                  m3u8  | audio only          unknown             Default
234 mp4   audio only        |                  m3u8  | audio only          unknown             Default
139 m4a   audio only      2 |  907.31KiB   49k https | audio only          mp4a.40.5   49k 22k low, m4a_dash
249 webm  audio only      2 |  958.14KiB   52k https | 

[youtube] Bf2IBH_U97o: Downloading m3u8 information
[info] Available formats for Bf2IBH_U97o:
ID  EXT   RESOLUTION FPS CH |   FILESIZE  TBR PROTO | VCODEC         VBR ACODEC      ABR ASR MORE INFO
--------------------------------------------------------------------------------------------------------------
sb3 mhtml 48x27        1    |                 mhtml | images                                 storyboard
sb2 mhtml 45x45        1    |                 mhtml | images                                 storyboard
sb1 mhtml 90x90        1    |                 mhtml | images                                 storyboard
sb0 mhtml 180x180      1    |                 mhtml | images                                 storyboard
233 mp4   audio only        |                 m3u8  | audio only         unknown             Default
234 mp4   audio only        |                 m3u8  | audio only         unknown             Default
139 m4a   audio only      2 |  985.69KiB  49k https | audio only         m

In [5]:
with open("./outputs/data/playlists.json", "w") as f:
    json.dump(info, f)

with open("./outputs/data/playlists.json", "r") as f:
    info = json.load(f)

In [6]:
len(info["entries"])

40

In [7]:
options = {
    # "listformats": True,
    "cookiefile": "./outputs/cookies.txt",
    "outtmpl": "./outputs/downloads/%(title)s.%(ext)s",  # Output file name
}

for i, entry in enumerate(info["entries"]):
    if not entry:
        print(f"{i + 1}. None")
        continue

    formats = [f for f in entry["formats"] if f["resolution"] == "audio only"]
    formats.sort(key=lambda x: x["tbr"] or 0, reverse=True)
    format_id = formats[0]["format_id"]

    title = entry["title"]
    title = title.replace("｜", "|")
    title = title.replace("：", ":")
    title = title.replace("＂", "\"",)
    title = title.replace("/", "-")
    
    output_file = "./outputs/downloads/" + title + "." + formats[0]["audio_ext"]
    options["outtmpl"] = output_file
    
    if os.path.exists(output_file):
        print(f"{i + 1}. {entry['title']} - EXISTED")
        continue
    print(f"{i + 1}. {entry['title']}")

    options["format"] = format_id
    with YoutubeDL(options) as ydl:
        ydl.download(entry["original_url"])

print("Download completed!")

1. Chơi Như Tụi Mỹ
[youtube] Extracting URL: https://music.youtube.com/watch?v=umKLy_E5Dyk
[youtube] umKLy_E5Dyk: Downloading webpage
[youtube] umKLy_E5Dyk: Downloading ios player API JSON
[youtube] umKLy_E5Dyk: Downloading mweb player API JSON
[youtube] umKLy_E5Dyk: Downloading ios music player API JSON
[youtube] umKLy_E5Dyk: Downloading web music client config
[youtube] umKLy_E5Dyk: Downloading player 03dbdfab
[youtube] umKLy_E5Dyk: Downloading web music player API JSON


[youtube] umKLy_E5Dyk: Downloading m3u8 information
[info] umKLy_E5Dyk: Downloading 1 format(s): 251
[download] Destination: ./outputs/downloads/Chơi Như Tụi Mỹ.webm
[download] 100% of    2.89MiB in 00:00:00 at 6.86MiB/s   
2. Đổi Tư Thế
[youtube] Extracting URL: https://music.youtube.com/watch?v=mFAodH3PjPo
[youtube] mFAodH3PjPo: Downloading webpage
[youtube] mFAodH3PjPo: Downloading ios player API JSON
[youtube] mFAodH3PjPo: Downloading mweb player API JSON
[youtube] mFAodH3PjPo: Downloading ios music player API JSON
[youtube] mFAodH3PjPo: Downloading web music client config
[youtube] mFAodH3PjPo: Downloading player 03dbdfab
[youtube] mFAodH3PjPo: Downloading web music player API JSON


[youtube] mFAodH3PjPo: Downloading m3u8 information
[info] mFAodH3PjPo: Downloading 1 format(s): 251
[download] Destination: ./outputs/downloads/Đổi Tư Thế.webm
[download] 100% of    4.07MiB in 00:00:00 at 9.50MiB/s   
3. Dân Chơi Sao Phải Khóc
[youtube] Extracting URL: https://music.youtube.com/watch?v=9zeE-ScuVNg
[youtube] 9zeE-ScuVNg: Downloading webpage
[youtube] 9zeE-ScuVNg: Downloading ios player API JSON
[youtube] 9zeE-ScuVNg: Downloading mweb player API JSON
[youtube] 9zeE-ScuVNg: Downloading ios music player API JSON
[youtube] 9zeE-ScuVNg: Downloading web music client config
[youtube] 9zeE-ScuVNg: Downloading player 03dbdfab
[youtube] 9zeE-ScuVNg: Downloading web music player API JSON


[youtube] 9zeE-ScuVNg: Downloading m3u8 information
[info] 9zeE-ScuVNg: Downloading 1 format(s): 251
[download] Destination: ./outputs/downloads/Dân Chơi Sao Phải Khóc.webm
[download] 100% of    3.37MiB in 00:00:00 at 8.14MiB/s   
4. Nhạc Anh (feat. Wxrdie)
[youtube] Extracting URL: https://music.youtube.com/watch?v=Izi3xp7Xd5Y
[youtube] Izi3xp7Xd5Y: Downloading webpage
[youtube] Izi3xp7Xd5Y: Downloading ios player API JSON
[youtube] Izi3xp7Xd5Y: Downloading mweb player API JSON
[youtube] Izi3xp7Xd5Y: Downloading ios music player API JSON
[youtube] Izi3xp7Xd5Y: Downloading web music client config
[youtube] Izi3xp7Xd5Y: Downloading player 03dbdfab
[youtube] Izi3xp7Xd5Y: Downloading web music player API JSON


[youtube] Izi3xp7Xd5Y: Downloading m3u8 information
[info] Izi3xp7Xd5Y: Downloading 1 format(s): 251
[download] Destination: ./outputs/downloads/Nhạc Anh (feat. Wxrdie).webm
[download] 100% of    2.94MiB in 00:00:00 at 6.81MiB/s   
5. DCME (feat. Rhymastic)
[youtube] Extracting URL: https://music.youtube.com/watch?v=RTUAHwXhUjU
[youtube] RTUAHwXhUjU: Downloading webpage
[youtube] RTUAHwXhUjU: Downloading ios player API JSON
[youtube] RTUAHwXhUjU: Downloading mweb player API JSON
[youtube] RTUAHwXhUjU: Downloading ios music player API JSON
[youtube] RTUAHwXhUjU: Downloading web music client config
[youtube] RTUAHwXhUjU: Downloading player 03dbdfab
[youtube] RTUAHwXhUjU: Downloading web music player API JSON


...

[youtube] XOIzYukFl0Q: Downloading m3u8 information
[info] XOIzYukFl0Q: Downloading 1 format(s): 251
[download] Destination: ./outputs/downloads/Chưa Xong.webm
[download] 100% of    4.51MiB in 00:00:00 at 7.77MiB/s   
37. Champion
[youtube] Extracting URL: https://music.youtube.com/watch?v=Ngww2sWyDpA
[youtube] Ngww2sWyDpA: Downloading webpage
[youtube] Ngww2sWyDpA: Downloading ios player API JSON
[youtube] Ngww2sWyDpA: Downloading mweb player API JSON
[youtube] Ngww2sWyDpA: Downloading ios music player API JSON
[youtube] Ngww2sWyDpA: Downloading web music client config
[youtube] Ngww2sWyDpA: Downloading player 03dbdfab
[youtube] Ngww2sWyDpA: Downloading web music player API JSON


[youtube] Ngww2sWyDpA: Downloading m3u8 information
[info] Ngww2sWyDpA: Downloading 1 format(s): 251
[download] Destination: ./outputs/downloads/Champion.webm
[download] 100% of    5.56MiB in 00:00:00 at 10.49MiB/s  
38. Vô Điều Kiện
[youtube] Extracting URL: https://music.youtube.com/watch?v=7UtbITFIzOE
[youtube] 7UtbITFIzOE: Downloading webpage
[youtube] 7UtbITFIzOE: Downloading ios player API JSON
[youtube] 7UtbITFIzOE: Downloading mweb player API JSON
[youtube] 7UtbITFIzOE: Downloading ios music player API JSON
[youtube] 7UtbITFIzOE: Downloading web music client config
[youtube] 7UtbITFIzOE: Downloading player 03dbdfab
[youtube] 7UtbITFIzOE: Downloading web music player API JSON


[youtube] 7UtbITFIzOE: Downloading m3u8 information
[info] 7UtbITFIzOE: Downloading 1 format(s): 251
[download] Destination: ./outputs/downloads/Vô Điều Kiện.webm
[download] 100% of    2.74MiB in 00:00:00 at 7.40MiB/s   
39. 1000 Ánh Mắt
[youtube] Extracting URL: https://music.youtube.com/watch?v=hXrr9iu1Ois
[youtube] hXrr9iu1Ois: Downloading webpage
[youtube] hXrr9iu1Ois: Downloading ios player API JSON
[youtube] hXrr9iu1Ois: Downloading mweb player API JSON
[youtube] hXrr9iu1Ois: Downloading ios music player API JSON
[youtube] hXrr9iu1Ois: Downloading web music client config
[youtube] hXrr9iu1Ois: Downloading player 03dbdfab
[youtube] hXrr9iu1Ois: Downloading web music player API JSON


[youtube] hXrr9iu1Ois: Downloading m3u8 information
[info] hXrr9iu1Ois: Downloading 1 format(s): 251
[download] Destination: ./outputs/downloads/1000 Ánh Mắt.webm
[download] 100% of    2.41MiB in 00:00:00 at 6.29MiB/s   
40. Track 06
[youtube] Extracting URL: https://music.youtube.com/watch?v=Bf2IBH_U97o
[youtube] Bf2IBH_U97o: Downloading webpage
[youtube] Bf2IBH_U97o: Downloading ios player API JSON
[youtube] Bf2IBH_U97o: Downloading mweb player API JSON
[youtube] Bf2IBH_U97o: Downloading ios music player API JSON
[youtube] Bf2IBH_U97o: Downloading web music client config
[youtube] Bf2IBH_U97o: Downloading player 03dbdfab
[youtube] Bf2IBH_U97o: Downloading web music player API JSON


[youtube] Bf2IBH_U97o: Downloading m3u8 information
[info] Bf2IBH_U97o: Downloading 1 format(s): 251
[download] Destination: ./outputs/downloads/Track 06.webm
[download] 100% of    2.74MiB in 00:00:00 at 6.90MiB/s   
Download completed!


In [11]:
not_exists = []
for i, entry in enumerate(info["entries"]):
    if not entry:
        print(f"{i + 1}. None")
        continue

    formats = [f for f in entry["formats"] if f["resolution"] == "audio only"]
    formats.sort(key=lambda x: x["tbr"] or 0, reverse=True)
    format_id = formats[0]["format_id"]

    title = entry["title"]
    title = title.replace("｜", "|")
    title = title.replace("：", ":")
    title = title.replace("＂", "\"",)
    title = title.replace("/", "-")

    fulltitle = entry["fulltitle"]
    fulltitle = fulltitle.replace("｜", "|")
    fulltitle = fulltitle.replace("：", ":")
    fulltitle = fulltitle.replace("＂", "\"",)
    fulltitle = fulltitle.replace("/", "-")

    webm = "./outputs/downloads/" + title + "." + formats[0]["audio_ext"]
    mp3 = "./outputs/thumbnails/" + fulltitle + "." + "mp3"
    # webm = webm.replace("|", "｜")
    # webm = webm.replace(":", "：")
    # webm = webm.replace("\"", "＂")
    # webm = webm.replace("/", "⧸")
    
    
    if not os.path.exists(webm):
        not_exists.append(webm)
        print(f"{i+1}. Not existed - ", webm)
        continue
    if os.path.exists(mp3):
        print(f"{i+1}. Existed - ", mp3)
        continue

    ffmpeg.input(webm).output(mp3, vn=None, acodec="libmp3lame", q=0).run()

    audiofile = eyed3.load(mp3)
    img_data = requests.get(entry["thumbnail"]).content
    audiofile.tag.images.set(3, img_data, "image/jpeg")
    audiofile.tag.artist = entry.get("artist", None)
    audiofile.tag.album = entry.get("album", None)
    audiofile.tag.title = entry.get("fulltitle", None)

    audiofile.tag.save()

    print(f"{i+1}. Thumbnail added to {mp3}")

1. Existed -  ./outputs/thumbnails/Chơi Như Tụi Mỹ.mp3
2. Existed -  ./outputs/thumbnails/Đổi Tư Thế.mp3
3. Existed -  ./outputs/thumbnails/Dân Chơi Sao Phải Khóc.mp3
4. Existed -  ./outputs/thumbnails/Nhạc Anh (feat. Wxrdie).mp3
5. Existed -  ./outputs/thumbnails/DCME (feat. Rhymastic).mp3
6. Existed -  ./outputs/thumbnails/Tim Anh Ghen.mp3
7. Existed -  ./outputs/thumbnails/420ChillnFree.mp3
8. Existed -  ./outputs/thumbnails/Đừng.mp3
9. Existed -  ./outputs/thumbnails/No Days Off.mp3
10. Existed -  ./outputs/thumbnails/Kỉ Nguyên Mới.mp3
11. Existed -  ./outputs/thumbnails/Nép Vào Sát Vào.mp3
12. Existed -  ./outputs/thumbnails/40 độ.mp3
13. Existed -  ./outputs/thumbnails/CHÊ HỘ.mp3
14. Existed -  ./outputs/thumbnails/Vinflow.mp3
15. Existed -  ./outputs/thumbnails/Mưa Cứ Rơi.mp3
16. Existed -  ./outputs/thumbnails/29.mp3
17. Existed -  ./outputs/thumbnails/Youngz (feat. Young Milo).mp3
18. Existed -  ./outputs/thumbnails/CAN GI NOI IU.mp3
19. Existed -  ./outputs/thumbnails/Mời Em.

In [9]:
len(not_exists), not_exists

(0, [])